# This is the final, end-to-end execution plan. I have updated both the **Data Processor** (to handle your specific JSON headers) and the **Streamlit Dashboard** (to match the new data structure).

### **Step 1: Create `data_processor_v3.py**`

This script converts your messy 1-million-row file into a clean, small ZIP file for the dashboard. It solves the "empty fields" issue by parsing the JSON categories correctly.

In [ ]:
import pandas as pd
import numpy as np
import re
import json

# ==========================================
# CONFIGURATION
# ==========================================
# 1. CHANGE THIS to your large file name
INPUT_FILE = "tech_talent_radar.csv"
OUTPUT_FILE = "tech_talent_radar_final.zip"

# 2. Define Tech Stack Keywords (The Strategy Engine)
TECH_KEYWORDS = {
    'Python': r'\bpython\b',
    'Java': r'\bjava\b',
    'React/JS': r'\b(react|node|javascript|typescript|vue|angular)\b',
    'Cloud/AWS': r'\b(aws|azure|cloud|gcp|google cloud)\b',
    'Data/AI': r'\b(data|ai|machine learning|nlp|torch|tensorflow|bi|tableau)\b',
    'Cybersecurity': r'\b(cyber|security|infosec)\b',
    'DevOps': r'\b(devops|sre|ci/cd|kubernetes|docker|jenkins)\b',
    '.NET/C#': r'\b(\.net|c#|dotnet)\b',
    'Civil/Struct': r'\b(civil|structural|tunnel|bridge|geotechnical)\b',
    'Mechanical': r'\b(mechanical|hvac|piping|m&e)\b',
    'Electrical': r'\b(electrical|power|switchgear)\b'
}

def parse_categories(val):
    """Extracts category names from JSON string."""
    try:
        # Handle string representation of list
        if isinstance(val, str) and val.strip().startswith('['):
            # Safe eval or json load
            val = val.replace("'", '"') # Fix common quote issues
            data = json.loads(val)
            return [item.get('category', '') for item in data]
        return []
    except:
        return []

def get_tech_stack(title):
    """Scans title for keywords to assign a 'Stack'."""
    title = str(title).lower()
    for stack, pattern in TECH_KEYWORDS.items():
        if re.search(pattern, title):
            return stack
    return None

def clean_salary(val):
    try:
        return float(val)
    except:
        return np.nan

def process():
    print(f"🔄 Loading {INPUT_FILE}...")
    try:
        df = pd.read_csv(INPUT_FILE, low_memory=False)
    except FileNotFoundError:
        print(f"❌ Error: Could not find {INPUT_FILE}. Please rename your large file.")
        return

    # 1. Filter for IT & Engineering (Using JSON parsing)
    print("   Parsing Categories & Filtering...")

    # Helper to filter rows
    def is_target(row_val):
        cats = parse_categories(row_val)
        return any(c in ['Information Technology', 'Engineering'] for c in cats)

    mask = df['categories'].apply(is_target)
    df_filtered = df[mask].copy()
    print(f"   ✅ Filtered down to {len(df_filtered)} rows.")

    # 2. Rename Columns to Standard Names
    print("   Standardizing Columns...")
    rename_map = {
        'title': 'job_title',
        'postedCompany_name': 'company',
        'metadata_newPostingDate': 'date',
        'average_salary': 'salary_avg',
        'salary_minimum': 'salary_min',
        'salary_maximum': 'salary_max'
    }
    df_filtered.rename(columns=rename_map, inplace=True)

    # 3. Create 'category' column (Simple string for dashboard)
    def get_main_cat(val):
        cats = parse_categories(val)
        if 'Information Technology' in cats: return 'Information Technology'
        if 'Engineering' in cats: return 'Engineering'
        return 'Other'
    df_filtered['category'] = df_filtered['categories'].apply(get_main_cat)

    # 4. Tag Tech Stacks
    print("   Tagging Tech Stacks...")
    df_filtered['Tech_Stack'] = df_filtered['job_title'].apply(get_tech_stack)

    # 5. Clean Salary
    for col in ['salary_min', 'salary_max', 'salary_avg']:
        if col in df_filtered.columns:
            df_filtered[col] = df_filtered[col].apply(clean_salary)

    # Fill missing average if min/max exist
    mask = df_filtered['salary_avg'].isna()
    df_filtered.loc[mask, 'salary_avg'] = (df_filtered['salary_min'] + df_filtered['salary_max']) / 2

    # 6. Save
    print(f"💾 Saving to {OUTPUT_FILE}...")
    cols = ['job_title', 'company', 'category', 'salary_avg', 'date', 'Tech_Stack']
    # Filter for existing columns only
    final_cols = [c for c in cols if c in df_filtered.columns]

    df_filtered[final_cols].to_csv(OUTPUT_FILE, index=False, compression='zip')
    print("✅ Done! Ready for Dashboard.")

if __name__ == "__main__":
    process()

---

### **Step 2: Create `app.py` (The Dashboard)**

This is the Streamlit app that reads the cleaned ZIP file.

In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px

# PAGE CONFIG
st.set_page_config(page_title="Tech Talent Radar", layout="wide")

# LOAD DATA
@st.cache_data
def load_data():
    # Load the compressed file directly
    df = pd.read_csv("tech_talent_radar_final.zip")
    # Convert date
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
    return df

try:
    df = load_data()
except FileNotFoundError:
    st.error("❌ Data file not found. Please run 'data_processor_v3.py' first!")
    st.stop()

# TITLE & SIDEBAR
st.title("📡 Tech & Engineering Talent Radar")
st.markdown("### Competitive Intelligence for Singapore's Tech Sector")

with st.sidebar:
    st.header("Filters")
    # Category Filter
    cats = df['category'].unique()
    selected_cat = st.radio("Select Sector:", cats)

    # Filter Data by Category first
    df_cat = df[df['category'] == selected_cat]

    # Tech Stack Filter
    stacks = df_cat['Tech_Stack'].dropna().unique()
    selected_stacks = st.multiselect("Filter Tech Stacks:", stacks, default=stacks[:5])

# APPLY FILTERS
if selected_stacks:
    df_filtered = df_cat[df_cat['Tech_Stack'].isin(selected_stacks)]
else:
    df_filtered = df_cat

# KPIS
c1, c2, c3 = st.columns(3)
c1.metric("Active Job Postings", len(df_filtered))
avg_sal = df_filtered['salary_avg'].mean()
c2.metric("Avg Monthly Salary", f"${avg_sal:,.0f}" if not pd.isna(avg_sal) else "N/A")
top_skill = df_filtered['Tech_Stack'].mode()[0] if not df_filtered.empty else "N/A"
c3.metric("Top In-Demand Skill", top_skill)

st.divider()

# TABS
tab1, tab2, tab3 = st.tabs(["💰 Value (Salary)", "📈 Demand (Volume)", "🏢 Competition (Strategy)"])

with tab1:
    st.subheader("Which Tech Stack Pays the Most?")
    if not df_filtered.empty:
        # Sort by median salary
        order = df_filtered.groupby('Tech_Stack')['salary_avg'].median().sort_values(ascending=False).index
        fig = px.box(df_filtered, x='Tech_Stack', y='salary_avg', color='Tech_Stack',
                     category_orders={'Tech_Stack': order}, points=False)
        st.plotly_chart(fig, use_container_width=True)
    else:
        st.info("Select Tech Stacks in the sidebar to see data.")

with tab2:
    st.subheader("Hiring Demand Over Time")
    if 'date' in df_filtered.columns and not df_filtered['date'].isna().all():
        # Group by Month
        trend = df_filtered.groupby([pd.Grouper(key='date', freq='M'), 'Tech_Stack']).size().reset_index(name='Count')
        fig2 = px.line(trend, x='date', y='Count', color='Tech_Stack', markers=True)
        st.plotly_chart(fig2, use_container_width=True)
    else:
        st.warning("Date data is missing or invalid.")

with tab3:
    st.subheader("Top Companies Hiring for these Roles")
    if not df_filtered.empty:
        top_companies = df_filtered['company'].value_counts().head(10).index
        df_top = df_filtered[df_filtered['company'].isin(top_companies)]

        fig3 = px.histogram(df_top, y='company', color='Tech_Stack', barmode='stack')
        st.plotly_chart(fig3, use_container_width=True)

---

### **Step 3: Run Instructions (Execute EXACTLY in order)**

1. **Prepare your Data File:**
* Take your original large file (`tech_talent_radar_sampleData...` or the 1M row file).
* **Rename it** to exactly: `tech_talent_radar.csv`
* Place it in a folder with `data_processor_v3.py` and `app.py`.


2. **Run the Processor:**
* Open your terminal/command prompt.
* Run: `python data_processor_v3.py`
* *Wait until it says "✅ Done!". It will create `tech_talent_radar_final.zip`.*


3. **Run the Dashboard:**
* Run: `streamlit run app.py`



This will launch your dashboard in the browser. It will now have accurate salaries, correct tech stacks (Python, Java, etc.), and working charts.